In [23]:
import pandas as pd
import numpy as np
import math
import heapq
import time
import folium
import ipywidgets as widgets
from IPython.display import display

# -------------------------
# 1. Load Cleaned Cities CSV
# -------------------------
cities_df = pd.read_csv("Data/Cities_of_SriLanka.csv")

# Convert to dictionary for easy lookup
city_coords = {row['name_en']: (row['latitude'], row['longitude']) for _, row in cities_df.iterrows()}

# -------------------------
# 2. Haversine Distance Function
# -------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# -------------------------
# 3. Build k-Nearest Graph
# -------------------------
K = 10  # number of neighbors
cities = list(city_coords.keys())
graph = {city: [] for city in cities}

for city in cities:
    lat1, lon1 = city_coords[city]
    distances = []
    for other in cities:
        if city == other:
            continue
        lat2, lon2 = city_coords[other]
        d = haversine(lat1, lon1, lat2, lon2)
        distances.append((d, other))
    distances.sort()
    for d, neighbor in distances[:K]:
        graph[city].append((neighbor, d))



In [24]:
# -------------------------
# 4. Shortest Path Algorithms
# -------------------------

def dijkstra(graph, start, end):
    pq = [(0, start, [])]
    visited = set()
    while pq:
        (dist, node, path) = heapq.heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        path = path + [node]
        if node == end:
            return dist, path
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                heapq.heappush(pq, (dist+weight, neighbor, path))
    return float("inf"), []


def bellman_ford(graph, start, end):
    dist = {node: float("inf") for node in graph}
    pred = {node: None for node in graph}
    dist[start] = 0
    
    for _ in range(len(graph)-1):
        for u in graph:
            for v, w in graph[u]:
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
                    pred[v] = u
    
    # Reconstruct path
    path = []
    node = end
    while node:
        path.insert(0, node)
        node = pred[node]
    return dist[end], path

# A* Algorithm
def a_star(graph, start, end):
    def heuristic(u, v):
        lat1, lon1 = city_coords[u]
        lat2, lon2 = city_coords[v]
        return haversine(lat1, lon1, lat2, lon2)
    
    open_set = [(0, start, [])]
    g_score = {node: float("inf") for node in graph}
    g_score[start] = 0
    f_score = {node: float("inf") for node in graph}
    f_score[start] = heuristic(start, end)
    visited = set()
    
    while open_set:
        (f, node, path) = heapq.heappop(open_set)
        if node in visited:
            continue
        visited.add(node)
        path = path + [node]
        if node == end:
            return g_score[end], path
        for neighbor, weight in graph[node]:
            tentative_g = g_score[node] + weight
            if tentative_g < g_score[neighbor]:
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, end)
                heapq.heappush(open_set, (f_score[neighbor], neighbor, path))
    return float("inf"), []


# def floyd_warshall(graph):
#     nodes = list(graph.keys())
#     idx = {node: i for i, node in enumerate(nodes)}
#     n = len(nodes)
#     dist = [[float("inf")] * n for _ in range(n)]
#     next_node = [[None]*n for _ in range(n)]
    
#     for u in graph:
#         i = idx[u]
#         dist[i][i] = 0
#         for v, w in graph[u]:
#             j = idx[v]
#             dist[i][j] = w
#             next_node[i][j] = v
    
#     for k in range(n):
#         for i in range(n):
#             for j in range(n):
#                 if dist[i][k] + dist[k][j] < dist[i][j]:
#                     dist[i][j] = dist[i][k] + dist[k][j]
#                     next_node[i][j] = next_node[i][k]
    
#     def get_path(u, v):
#         if next_node[idx[u]][idx[v]] is None:
#             return []
#         path = [u]
#         while u != v:
#             u = next_node[idx[u]][idx[v]]
#             path.append(u)
#         return path
    
#     return dist, get_path, nodes

# -------------------------
# 5. User Input & Run Example
# -------------------------
# Create interactive dropdowns for source and destination city selection
source_dd = widgets.Combobox(
    placeholder='Select source city',
    options=sorted(list(city_coords.keys())),
    description='Source City:',
    ensure_option=True,
    disabled=False
)

destination_dd = widgets.Combobox(
    placeholder='Select destination city',
    options=sorted(list(city_coords.keys())),
    description='Destination City:',
    ensure_option=True,
    disabled=False
)

# Display the dropdowns
print("Select the source and destination cities:")
display(source_dd, destination_dd)

# Wait for user to select cities



Select the source and destination cities:


Combobox(value='', description='Source City:', ensure_option=True, options=('Aandankulam', 'Adampan', 'Addalai…

Combobox(value='', description='Destination City:', ensure_option=True, options=('Aandankulam', 'Adampan', 'Ad…

In [25]:
run_button = widgets.Button(description="Find Shortest Path")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        source = source_dd.value
        destination = destination_dd.value
        if source not in city_coords or destination not in city_coords:
            print("Please select valid cities from the dropdowns.")
            return
        
        print(f"Finding shortest path from {source} to {destination}\n")

        # Dijkstra
        start_time = time.time()
        dist, path = dijkstra(graph, source, destination)
        elapsed = round(time.time()-start_time, 6)
        print(f"Dijkstra -> Distance: {round(dist,2)} km | Time: {elapsed} s")

        # Bellman-Ford
        start_time = time.time()
        dist, path = bellman_ford(graph, source, destination)
        elapsed = round(time.time()-start_time, 6)
        print(f"Bellman-Ford -> Distance: {round(dist,2)} km | Time: {elapsed} s")

        

        # -------------------------
        # 6. Folium Visualization
        # -------------------------
        
        if path:
            lat, lon = city_coords[source]
            m = folium.Map(location=[lat, lon], zoom_start=8)

            # Add cities to map
            for city in path:
                lat, lon = city_coords[city]
                folium.Marker([lat, lon], popup=city, icon=folium.Icon(color="blue")).add_to(m)

            # Draw path line
            coords = [(city_coords[city][0], city_coords[city][1]) for city in path]
            folium.PolyLine(coords, color="red", weight=3, opacity=0.8).add_to(m)

            display(m)   # ⬅️ Shows map inline in Jupyter

run_button.on_click(on_button_clicked)

display(run_button, output)

Button(description='Find Shortest Path', style=ButtonStyle())

Output()